In [ ]:
# %pip install prefect_gcp
# %pip install prefect_github

   ---------------------------------------- 0.0/13.7 MB ? eta -:--:--
   - -------------------------------------- 0.5/13.7 MB 3.2 MB/s eta 0:00:05
   --- ------------------------------------ 1.3/13.7 MB 3.4 MB/s eta 0:00:04
   ------ --------------------------------- 2.1/13.7 MB 3.7 MB/s eta 0:00:04
   --------- ------------------------------ 3.1/13.7 MB 3.9 MB/s eta 0:00:03
   ----------- ---------------------------- 3.9/13.7 MB 3.9 MB/s eta 0:00:03
   ------------- -------------------------- 4.7/13.7 MB 3.9 MB/s eta 0:00:03
   ---------------- ----------------------- 5.8/13.7 MB 4.1 MB/s eta 0:00:02
   -------------------- ------------------- 7.1/13.7 MB 4.3 MB/s eta 0:00:02
   ----------------------- ---------------- 8.1/13.7 MB 4.5 MB/s eta 0:00:02
   ---------------------------- ----------- 9.7/13.7 MB 4.7 MB/s eta 0:00:01
   -------------------------------- ------- 11.0/13.7 MB 4.9 MB/s eta 0:00:01
   ------------------------------------ --- 12.3/13.7 MB 5.1 MB/s eta 0:00:01
   -

In [ ]:
# %pip install prefect

   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   --- ------------------------------------ 0.5/6.1 MB 2.9 MB/s eta 0:00:02
   ------ --------------------------------- 1.0/6.1 MB 2.7 MB/s eta 0:00:02
   ------------ --------------------------- 1.8/6.1 MB 3.0 MB/s eta 0:00:02
   --------------- ------------------------ 2.4/6.1 MB 3.0 MB/s eta 0:00:02
   ------------------- -------------------- 2.9/6.1 MB 3.0 MB/s eta 0:00:02
   ------------------------ --------------- 3.7/6.1 MB 3.0 MB/s eta 0:00:01
   --------------------------- ------------ 4.2/6.1 MB 3.0 MB/s eta 0:00:01
   ------------------------------- -------- 4.7/6.1 MB 3.0 MB/s eta 0:00:01
   ------------------------------------ --- 5.5/6.1 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 6.1/6.1 MB 3.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 2.7 MB/s eta 0:00:01
   -----------------------

In [ ]:
# %pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Atur Timezone
# %pip install pytz

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Untuk melakukan Geocoding
# %pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [11]:
# Import Library

import pandas as pd
import numpy as np
import pandas_gbq as gbq
import requests
import logging

# Datetime and Timezone Library
from datetime import datetime, timezone
from pytz import timezone as tz

# Geocoding Library
from geopy.geocoders import Nominatim

# Google API Library
from google.oauth2 import service_account

# Setup logger
logging.basicConfig(level=logging.INFO)

Table of Content
1. Extract
2. Transform
3. Load

# 1. Data Extraction

In [12]:
# Define Base_URL

# BASE_URL for real-time earthquake data (past 24 hours)
BASE_URL = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson'

# BASE_URL for historical earthquake data
# BASE_URL = 'https://earthquake.usgs.gov/fdsnws/event/1/query'

In [ ]:
requests.get(BASE_URL)

<Response [200]>

In [17]:
# Get_all real-time earthquake data

reponse = requests.get(BASE_URL)
data = reponse.json()

In [20]:
data['features'][0]['geometry']

{'type': 'Point', 'coordinates': [-116.029, 33.782, 1.04]}

In [23]:
# Convert JSON to Dataframe

records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)

    records_df = pd.DataFrame(records)

In [24]:
records_df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,ci41017575,"18 km NE of Coachella, CA",1.23,2025-07-09 03:30:23.770,2025-07-09 03:33:52.187,None,NaN,NaN,NaN,None,...,23,ci,41017575,",ci41017575,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",-116.029000,33.782000,1.040,2025-07-09 03:42:19.417608+00:00
1,nc75207327,"2 km E of Pinnacles, CA",1.87,2025-07-09 03:27:19.310,2025-07-09 03:30:28.520,None,NaN,NaN,NaN,None,...,54,nc,75207327,",nc75207327,",",nc,",",focal-mechanism,nearby-cities,origin,phase-data,",-121.122833,36.535999,9.200,2025-07-09 03:42:19.424622+00:00
2,nc75207322,"8 km NNW of The Geysers, CA",0.68,2025-07-09 03:22:44.550,2025-07-09 03:24:19.610,None,NaN,NaN,NaN,None,...,7,nc,75207322,",nc75207322,",",nc,",",nearby-cities,origin,phase-data,",-122.815002,38.828499,1.410,2025-07-09 03:42:19.426902+00:00
3,uw62136042,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:17:23.080,2025-07-09 03:18:56.910,None,NaN,NaN,NaN,None,...,2,uw,62136042,",uw62136042,",",uw,",",origin,phase-data,",-121.768166,46.846832,-1.720,2025-07-09 03:42:19.428499+00:00
4,nc75207317,"4 km NNW of The Geysers, CA",0.75,2025-07-09 03:17:09.340,2025-07-09 03:18:44.596,None,NaN,NaN,NaN,None,...,9,nc,75207317,",nc75207317,",",nc,",",nearby-cities,origin,phase-data,",-122.778999,38.805168,2.370,2025-07-09 03:42:19.430239+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,nc75206902,"4 km NNW of The Geysers, CA",1.03,2025-07-08 04:11:39.870,2025-07-08 04:13:13.724,None,NaN,NaN,NaN,None,...,16,nc,75206902,",nc75206902,",",nc,",",nearby-cities,origin,phase-data,",-122.783997,38.803833,3.180,2025-07-09 03:42:20.507661+00:00
451,av93661711,"7 km W of Akutan, Alaska",0.64,2025-07-08 04:10:43.050,2025-07-08 21:22:40.670,None,NaN,NaN,NaN,None,...,6,av,93661711,",av93661711,",",av,",",origin,phase-data,",-165.888333,54.143000,4.810,2025-07-09 03:42:20.510277+00:00
452,us7000qba0,"29 km NNW of Diphu, India",4.20,2025-07-08 03:52:10.603,2025-07-08 04:08:33.040,None,NaN,NaN,NaN,None,...,271,us,7000qba0,",us7000qba0,",",us,",",origin,phase-data,",93.361700,26.102100,10.000,2025-07-09 03:42:20.513033+00:00
453,nc75206892,"6 km WSW of Gilroy, CA",1.47,2025-07-08 03:52:05.750,2025-07-08 04:12:20.273,None,NaN,NaN,NaN,None,...,33,nc,75206892,",nc75206892,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",-121.622833,36.972668,3.700,2025-07-09 03:42:20.516812+00:00


# 2. Transformation

- Basic Cleaning
- Data Enrichment

In [25]:
df = records_df.copy()

In [26]:
# Drop Data Duplicated
df = df.drop_duplicates(subset=['id'])

# Drop rows with null magnitude or location
df = df.dropna(subset=['mag', 'place', 'latitude', 'longitude', 'time'])

In [27]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [28]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [29]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [34]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=3)
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [35]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [36]:
# Save Data to CSV
df.to_csv(f"../data/earthquakes-{datetime.now().strftime('%Y%m%d')}.csv", index=False)

In [43]:
# Bigquery configuration
PROJECT_ID = 'precise-tenure-465304-d8'
TABLE_ID = 'earthquake.event' # Dataset.id
CREDENTIALS_PATH = '../config/precise-tenure-465304-d8-5da16d81a6ee.json'

In [44]:
credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_PATH)

In [47]:
gbq.to_gbq(
    dataframe = df, 
    destination_table = TABLE_ID, 
    project_id = PROJECT_ID, 
    credentials = credentials, 
    if_exists="replace")

425 out of 425 rows loaded.<?, ?it/s]INFO:pandas_gbq.gbq_connector:
100%|██████████| 1/1 [00:00<00:00, 535.12it/s]
